# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
np.random.seed(3524)

In [2]:
#necessary functions:
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""
    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1)-np.mean(data_2)
    return diff

def draw_bs_reps(data, groupsize, func, size=1):
    """Takes bootstrap replicate[s] of a sample"""
    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)
    # Generate replicates
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, groupsize))
    return bs_replicates
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1,data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2
def draw_perm_reps(data1, data2, func, size=1):
    """Generate multiple permutation replicates."""
    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)
    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data1, data2)
        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
    return perm_replicates

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)
print(data.race.value_counts())
print(data.call.value_counts())

w    2435
b    2435
Name: race, dtype: int64
0.0    4478
1.0     392
Name: call, dtype: int64


In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


1. Since we are looking at the proportion of callback rates between applicants with black-sounding names and those with white-sounding names, a two-sample z-test for the difference of proportions would be appropriate here. The central limit theorem would apply, since the sample was random, the sample size is large enough at over 4000 for us to assume normality, and that sample size is also definitely less than 10% of the total job-seeking population, making it an independent sample.
2. Our null hypothesis is that the difference between the population proportion of black people who received callbacks is equal to the population proportion of white people who received callbacks.

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

3. For our hypothesis test, we first use the frequentist statistical approach to find the margin of error, confidence interval, and p-value. Our significance level is 0.05; if our p-value is smaller than this, we will reject the null hypothesis. 

In [9]:
w_total=len(w)
b_total=len(b)
w_prop=np.mean(w.call) #proportion of white applicants with callbacks
b_prop=np.mean(b.call) #proportion of black candidates with callbacks
w_var=(w_prop*(1-w_prop))/w_total
b_var=(b_prop*(1-b_prop))/b_total
mean_proportion_diff=w_prop-b_prop
print("Mean difference in proportions: "+str(mean_proportion_diff))
std_proportion_diff=np.sqrt(w_var+b_var)
# Our margin of error is for a 95% confidence interval, so we need to find the z-value that we multiply by the 
# standard deviation of the proportion difference. 
z_star=stats.norm.ppf(.975)
MoE=z_star*std_proportion_diff
print("margin of error: "+str(np.round(MoE,8)))
conf_interval=np.array([mean_proportion_diff-MoE,mean_proportion_diff+MoE]) #look up formula
print("95% confidence interval: "+str(conf_interval))

# Assume null is true - assume that w_prop = b_prop, 
prop=sum(data.call)/len(data)
std_samp_dist=np.sqrt(prop*(1-prop)/len(w) + prop*(1-prop)/len(b))
z_0=mean_proportion_diff/std_samp_dist
p_val=stats.norm.pdf(z_0)*2
print("p value: "+str(np.round(p_val,8)))

Mean difference in proportions: 0.032032855
margin of error: 0.01525513
95% confidence interval: [0.01677773 0.04728798]
p value: 0.00017246


We now use the bootstrap approach to find the same values as above. Our test statistic is the difference in proportions of callbacks between white and black applicants. We will be taking 10000 permutation replicates of that stat to test the null hypothesis, and then will take 10000 bootstrap replicates of callback proportions to find the 95% confidence interval.

In [8]:
#compute 10000 permutation replicates for 
perm_reps=draw_perm_reps(w.call, b.call, diff_of_means,size=10000)
p_perm = np.sum(perm_reps >= mean_proportion_diff) / len(perm_reps)

#compute 10000 bs replicates of callback proportions for white and black applicants
bs_reps_w=draw_bs_reps(w.call,len(w),np.mean,10000)
bs_reps_b=draw_bs_reps(b.call,len(b),np.mean,10000)
bs_reps_diff=bs_reps_w-bs_reps_b
MoE_bs=bs_reps_diff.std()*stats.norm.ppf(.975)
bs_conf_int=np.percentile(bs_reps_diff,[2.5,97.5])
print("p value: "+str(p_perm))
print("Margin of error: "+str(MoE_bs))
print("95% confidence interval: "+str(bs_conf_int))

p value: 0.0
Margin of error: 0.015212563306275198
95% confidence interval: [0.01642711 0.04681725]


Both the bootstrap and the frequentist approaches gave p-values that were well below the .05 threshold, which would lead us to reject the null hypothesis that there is no actual difference in callback rates between black and white applicants. Additionally, the 95% confidence intervals returned do not contain the value 0, making it very unlikely that the true difference of proportions is 0.

4. Although the US government has taken steps to combat and prohibit racial discrimination in society, there still exists a significant amount of racial disparity in several aspects of life, including hiring practices. According to a randomized field experiment conducted in Chicago and Boston between 2000 and 2002, job applicants with "white-sounding" names were more likely to receive callbacks than those with "black-sounding" names, with the difference in percentage of callbacks between the two groups being 3.2%, with a 95% margin of error of 1.52%. This means that it is highly unlikely that the disparity in hiring practices observed between the two groups is due to random chance.
5. Since race was the only variable that was randomized in the study (with names being assigned randomly to resumes of identical quality), it would be fair to say that race is the most highly correlated factor in callback success for this study. However, because the study was limited to just two cities, and other aspects of the resumes weren't being tested for (such as formatting, length, etc), it cannot be determined whether race is the most important factor in callback success in general. If this test were conducted again, it would be useful to have other independent variables tested in addition to race, and to conduct it in more cities. 